In [43]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.alert import Alert
from datetime import datetime, timedelta
import pandas as pd
import time

# 파일 경로 설정
file_path = 'KOBIS_일자별_통계정보_2024-05-31.xls'

df = pd.read_csv('chunk_6_ex.csv')
# 컬럼 추가 및 초기화
df['fd_rank'] = None
df['w1_rank'] = None
df['w2_rank'] = None
df['w1_au'] = None
df['w2_au'] = None
df['w3_au'] = None

options = webdriver.ChromeOptions()
options.add_argument("chrome-win64/chromedriver.exe")
driver = webdriver.Chrome(options=options)



def check_date(date, df):
    if date in df['날짜'].values:
        screen_count = df[df['날짜'] == date]['스크린수'].values[0]
        if screen_count <= 9:
            print("screen value:", df[df['날짜'] == date]['스크린수'].values[0])
            return "no"
        else:
            return "yes"
    else:
        print("No date")
        return "no"

for index, row in df.iterrows():
    movie_title = row['Movie_Title']
    release_date = datetime.strptime(row['Release_Date'], '%Y-%m-%d')
    movie_code = row['movieCd']
    
 # 개봉일 전후 1주일 기간 설정
    week1_end_date = (release_date + timedelta(days=6)).strftime('%Y-%m-%d')
    week2_end_date = (release_date + timedelta(days=13)).strftime('%Y-%m-%d')
    week3_end_date = (release_date + timedelta(days=20)).strftime('%Y-%m-%d')
    start_date = release_date.strftime('%Y-%m-%d')

    # 웹 페이지 열기
    driver.get("https://www.kobis.or.kr/kobis/business/mast/mvie/searchMovieList.do")  # 실제 페이지 URL로 변경

    # 자바스크립트 함수 실행
    driver.execute_script(f"dtlExcelDn('movie','box','{movie_code}');")


    # 대화 상자 처리 (확인 버튼 클릭)
    time.sleep(3)  # 대화 상자가 뜨는 시간을 기다림
    alert = Alert(driver)
    alert.accept()  # 확인 버튼 클릭

    # 잠시 대기
    time.sleep(10)

    # HTML 파일 읽기
    tables = pd.read_html(file_path, encoding='utf-8')

    # 첫 번째 테이블을 데이터프레임으로 변환
    temp_df = tables[1]

    deci= check_date(week3_end_date, temp_df)
    if deci == "no":
        df = df[df['Movie_Title'] != movie_title]
        print(index, movie_title,"removed")
    else:
        df.at[index, 'fd_rank'] = temp_df[temp_df['날짜'] == start_date]['순위'].values[0]
        df.at[index, 'w1_rank'] = temp_df[temp_df['날짜'] == week1_end_date]['순위'].values[0]
        df.at[index, 'w2_rank'] = temp_df[temp_df['날짜'] == week2_end_date]['순위'].values[0]
        df.at[index, 'w1_au'] = temp_df[temp_df['날짜'] == week1_end_date]['누적관객수'].values[0]
        df.at[index, 'w2_au'] = temp_df[temp_df['날짜'] == week2_end_date]['누적관객수'].values[0]
        df.at[index, 'w3_au'] = temp_df[temp_df['날짜'] == week3_end_date]['누적관객수'].values[0]
        print(index, movie_title,"done!")

    if os.path.exists(file_path):
        os.remove(file_path)
        print(f"{file_path} 파일이 삭제되었습니다.")
    else:
        print(f"{file_path} 파일이 존재하지 않습니다.")

        
        
        
# Save the DataFrame to an Excel file
cel_filename = "preprosessing_ver1_chunk_6_ex.csv"
df.to_csv(cel_filename, index=False)



No date
0 플레인 removed
KOBIS_일자별_통계정보_2024-05-31.xls 파일이 삭제되었습니다.


In [ ]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.alert import Alert
from datetime import datetime, timedelta
import pandas as pd
import time

# 파일 경로 설정
file_path = 'KOBIS_일자별_통계정보_2024-06-03.xls'

df = pd.read_csv('chunk_6_ex.csv')
# 컬럼 추가 및 초기화
df['w3_rank'] = None
df['w1_av_sales']= None
df['w2_av_sales']= None
df['w3_av_sales']= None

options = webdriver.ChromeOptions()
options.add_argument("chrome-win64/chromedriver.exe")
driver = webdriver.Chrome(options=options)

def check_date(date, df):
    if date in df['날짜'].values:
        screen_count = df[df['날짜'] == date]['스크린수'].values[0]
        if screen_count <= 9:
            print("screen value:", df[df['날짜'] == date]['스크린수'].values[0])
            return "no"
        else:
            return "yes"
    else:
        print("No date")
        return "no"

def calculate_average_sales(temp_df, start_date, end_date):
    date_range = (temp_df['날짜'] >= start_date) & (temp_df['날짜'] <= end_date)
    sales = temp_df[date_range]['좌석판매율'].str.replace('%', '').astype(float)
    return sales.mean()

for index, row in df.iterrows():
    movie_title = row['Movie_Title']
    release_date = datetime.strptime(row['Release_Date'], '%Y-%m-%d')
    movie_code = row['movieCd']
    
    # 개봉일 전후 1주일 기간 설정
    week1_end_date = (release_date + timedelta(days=6)).strftime('%Y-%m-%d')
    week2_end_date = (release_date + timedelta(days=13)).strftime('%Y-%m-%d')
    week3_end_date = (release_date + timedelta(days=20)).strftime('%Y-%m-%d')
    start_date = release_date.strftime('%Y-%m-%d')

    # 웹 페이지 열기
    driver.get("https://www.kobis.or.kr/kobis/business/mast/mvie/searchMovieList.do")  # 실제 페이지 URL로 변경

    # 자바스크립트 함수 실행
    driver.execute_script(f"dtlExcelDn('movie','box','{movie_code}');")

    # 대화 상자 처리 (확인 버튼 클릭)
    time.sleep(3)  # 대화 상자가 뜨는 시간을 기다림
    alert = Alert(driver)
    alert.accept()  # 확인 버튼 클릭

    # 잠시 대기
    time.sleep(10)

    # HTML 파일 읽기
    tables = pd.read_html(file_path, encoding='utf-8')

    # 첫 번째 테이블을 데이터프레임으로 변환
    temp_df = tables[1]

    deci= check_date(week3_end_date, temp_df)
    if deci == "no":
        df = df[df['Movie_Title'] != movie_title]
        print(index, movie_title,"removed")
    else:
        df.at[index, 'w3_rank'] = temp_df[temp_df['날짜'] == week3_end_date]['순위'].values[0]
        
        # 좌석판매율 평균 계산 및 추가
        df.at[index, 'w1_av_sales'] = calculate_average_sales(temp_df, start_date, week1_end_date)
        df.at[index, 'w2_av_sales'] = calculate_average_sales(temp_df, (release_date + timedelta(days=7)).strftime('%Y-%m-%d'), week2_end_date)
        df.at[index, 'w3_av_sales'] = calculate_average_sales(temp_df, (release_date + timedelta(days=14)).strftime('%Y-%m-%d'), week3_end_date)
        
        print(index, movie_title,"done!")

    if os.path.exists(file_path):
        os.remove(file_path)
        print(f"{file_path} 파일이 삭제되었습니다.")
    else:
        print(f"{file_path} 파일이 존재하지 않습니다.")

# Save the DataFrame to an Excel file
csv_filename = "preprosessing_ver1_chunk_6_ex.csv"
df.to_csv(csv_filename, index=False)


In [1]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.alert import Alert
from datetime import datetime, timedelta
import pandas as pd
import time

# 파일 경로 설정
file_path = 'KOBIS_일자별_통계정보_2024-05-31.xls'

df = pd.read_csv('chunk_6_ex.csv')
# 컬럼 추가 및 초기화
df['w1_av_sc']=None
df['w2_av_sc']=None
df['w3_av_sc']=None

options = webdriver.ChromeOptions()
options.add_argument("chrome-win64/chromedriver.exe")
driver = webdriver.Chrome(options=options)



def check_date(date, df):
    if date in df['날짜'].values:
        screen_count = df[df['날짜'] == date]['스크린수'].values[0]
        if screen_count <= 9:
            print("screen value:", df[df['날짜'] == date]['스크린수'].values[0])
            return "no"
        else:
            return "yes"
    else:
        print("No date")
        return "no"
def calculate_average_screens(start_date, end_date, df):
    mask = (df['날짜'] >= start_date) & (df['날짜'] <= end_date)
    return df.loc[mask, '스크린수'].mean()


for index, row in df.iterrows():
    movie_title = row['Movie_Title']
    release_date = datetime.strptime(row['Release_Date'], '%Y-%m-%d')
    movie_code = row['movieCd']
    
 # 개봉일 전후 1주일 기간 설정
    week1_end_date = (release_date + timedelta(days=6)).strftime('%Y-%m-%d')
    week2_end_date = (release_date + timedelta(days=13)).strftime('%Y-%m-%d')
    week3_end_date = (release_date + timedelta(days=20)).strftime('%Y-%m-%d')
    start_date = release_date.strftime('%Y-%m-%d')

    week1_end_date_a = (release_date + timedelta(days=7)).strftime('%Y-%m-%d')
    week2_end_date_a = (release_date + timedelta(days=14)).strftime('%Y-%m-%d')
    week3_end_date_a = (release_date + timedelta(days=20)).strftime('%Y-%m-%d')

    # 웹 페이지 열기
    driver.get("https://www.kobis.or.kr/kobis/business/mast/mvie/searchMovieList.do")  # 실제 페이지 URL로 변경

    # 자바스크립트 함수 실행
    driver.execute_script(f"dtlExcelDn('movie','box','{movie_code}');")


    # 대화 상자 처리 (확인 버튼 클릭)
    time.sleep(3)  # 대화 상자가 뜨는 시간을 기다림
    alert = Alert(driver)
    alert.accept()  # 확인 버튼 클릭

    # 잠시 대기
    time.sleep(10)

    # HTML 파일 읽기
    tables = pd.read_html(file_path, encoding='utf-8')

    # 첫 번째 테이블을 데이터프레임으로 변환
    temp_df = tables[1]

    deci= check_date(week3_end_date, temp_df)
    if deci == "no":
        df = df[df['Movie_Title'] != movie_title]
        print(index, movie_title,"removed")
    else:
        df.at[index, 'w1_av_sc'] = calculate_average_screens(start_date, week1_end_date, temp_df) #1~7
        df.at[index, 'w2_av_sc'] = calculate_average_screens(week1_end_date_a, week2_end_date, temp_df) #8~14
        df.at[index, 'w3_av_sc'] = calculate_average_screens(week2_end_date_a, week3_end_date, temp_df) #15~21
        print(index, movie_title,"done!")

    if os.path.exists(file_path):
        os.remove(file_path)
        print(f"{file_path} 파일이 삭제되었습니다.")
    else:
        print(f"{file_path} 파일이 존재하지 않습니다.")

        
        
        
# Save the DataFrame to an Excel file
cel_filename = "preprosessing_ver1_chunk_6_ex.csv"
df.to_csv(cel_filename, index=False)



FileNotFoundError: [Errno 2] No such file or directory: 'updated_selected_columns_ver2.csv'

In [2]:
import pandas as pd

# CSV 파일 경로 설정
file_path = '0530_dataset/merged_dataset_0605.csv'

# CSV 파일 읽기
df = pd.read_csv(file_path)

# 행의 개수 출력
num_rows = len(df)
print(f'행의 개수: {num_rows}')


행의 개수: 387


In [2]:
import pandas as pd

# CSV 파일 경로 설정
file_path = '0530_dataset/combined_trend_data_adjusted.csv'

# CSV 파일 읽기
df = pd.read_csv(file_path)

# 총 행 개수
total_rows = len(df)

# 분할할 행 개수
chunk_size = 100

# CSV 파일을 나눌 디렉토리 설정
output_dir = 'output_chunks'

# 디렉토리 생성
import os
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 분할된 파일 저장
for i in range(0, total_rows, chunk_size):
    chunk = df.iloc[i:i + chunk_size]
    output_file = os.path.join(output_dir, f'chunk_{i // chunk_size + 1}.csv')
    chunk.to_csv(output_file, index=False)

print(f'총 {total_rows}개의 행을 {chunk_size}개씩 나눠서 CSV 파일로 저장했습니다.')


총 504개의 행을 100개씩 나눠서 CSV 파일로 저장했습니다.


In [19]:
import pandas as pd

# CSV 파일 경로 설정
file_path = 'chunk_1.csv'

# CSV 파일 읽기
df = pd.read_csv(file_path)

# 행의 개수 출력
num_rows = len(df)
print(f'행의 개수: {num_rows}')

행의 개수: 50


In [ ]:
import pandas as pd

# 첫 번째 CSV 파일 읽기
df1 = pd.read_csv('preprosessing_ver1_chunk_1.csv')

# 두 번째 CSV 파일 읽기
df2 = pd.read_csv('preprosessing_ver1_chunk_2.csv')

# 첫 번째 CSV 파일 읽기
df3 = pd.read_csv('preprosessing_ver1_chunk_3.csv')

# 두 번째 CSV 파일 읽기
df4 = pd.read_csv('preprosessing_ver1_chunk_4.csv')

# 데이터프레임을 수직으로 결합하기
result_1 = pd.concat([df1, df2], axis=0)

result_2 = pd.concat([df3, df4], axis=0)

result = pd.concat([result_1, result_2], axis=0)


# 결합된 데이터프레임을 새로운 CSV 파일로 저장하기
result.to_csv('merged_file.csv', index=False)
